In [1]:
import javalang
import itertools

from pipe import *

Attributes:
* method body
* return type
* argument list
* containing class name
* method name (label)

Attribute extensions:

In [8]:
code_snippet = """/*
 * Licensed to Elasticsearch under one or more contributor
 * license agreements. See the NOTICE file distributed with
 * this work for additional information regarding copyright
 * ownership. Elasticsearch licenses this file to you under
 * the Apache License, Version 2.0 (the "License"); you may
 * not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *    http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing,
 * software distributed under the License is distributed on an
 * "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
 * KIND, either express or implied.  See the License for the
 * specific language governing permissions and limitations
 * under the License.
 */

package org.elasticsearch.ingest.useragent;

import org.elasticsearch.common.cache.Cache;
import org.elasticsearch.common.cache.CacheBuilder;
import org.elasticsearch.ingest.useragent.UserAgentParser.Details;

import java.util.Objects;

class UserAgentCache {
    private final Cache<CompositeCacheKey, Details> cache;

    UserAgentCache(long cacheSize) {
        cache = CacheBuilder.<CompositeCacheKey, Details>builder().setMaximumWeight(cacheSize).build();
    }

    public Details get(String parserName, String userAgent) {
        return cache.get(new CompositeCacheKey(parserName, userAgent));
    }

    public void put(String parserName, String userAgent, Details details) {
        cache.put(new CompositeCacheKey(parserName, userAgent), details);
    }

    private static final class CompositeCacheKey {
        private final String parserName;
        private final String userAgent;

        CompositeCacheKey(String parserName, String userAgent) {
            this.parserName = parserName;
            this.userAgent = userAgent;
        }

        @Override
        public boolean equals(Object obj) {
            if(obj != null && obj instanceof CompositeCacheKey) {
                CompositeCacheKey s = (CompositeCacheKey)obj;
                return parserName.equals(s.parserName) && userAgent.equals(s.userAgent);
            }
            return false;
        }

        @Override
        public int hashCode() {
            return Objects.hash(parserName, userAgent);
        }
    }
}
"""

In [9]:
tree = javalang.parse.parse(code_snippet)
tree

CompilationUnit(imports=[Import(path=org.elasticsearch.common.cache.Cache, static=False, wildcard=False), Import(path=org.elasticsearch.common.cache.CacheBuilder, static=False, wildcard=False), Import(path=org.elasticsearch.ingest.useragent.UserAgentParser.Details, static=False, wildcard=False), Import(path=java.util.Objects, static=False, wildcard=False)], package=PackageDeclaration(annotations=None, documentation=None, modifiers=None, name=org.elasticsearch.ingest.useragent), types=[ClassDeclaration(annotations=[], body=[FieldDeclaration(annotations=[], declarators=[VariableDeclarator(dimensions=[], initializer=None, name=cache)], documentation=None, modifiers={'final', 'private'}, type=ReferenceType(arguments=[TypeArgument(pattern_type=None, type=ReferenceType(arguments=None, dimensions=[], name=CompositeCacheKey, sub_type=None)), TypeArgument(pattern_type=None, type=ReferenceType(arguments=None, dimensions=[], name=Details, sub_type=None))], dimensions=[], name=Cache, sub_type=None

In [30]:
def get_matching_closing_bracket_position(tokens):
    brackets = tokens | where(lambda token: token.value == '{' or token.value == '}')
    started_counting = False
    counter = 0
    end_position = None
    
    for bracket in brackets:
        if bracket.value == '{':
            started_counting = True
            counter += 1
        elif bracket.value == '}':
            counter -= 1

        if started_counting and counter == 0:
            end_position = bracket.position
            break
    
    return end_position

def get_method_body(tokens, line, column):
    filtered_tokens = tokens \
        | skip_while(lambda token: token.position.line < line) \
        | skip_while(lambda token: token.position.column < column) \
        | skip_while(lambda token: token.value != '{')
    
    # TODO: can we omit the generator copying
    filtered_tokens, filtered_tokens_copy = itertools.tee(filtered_tokens)
    closing_bracket_position = get_matching_closing_bracket_position(filtered_tokens_copy)
    
    body_tokens = filtered_tokens \
        | take_while(lambda token: token.position.line <= closing_bracket_position.line or \
                                   (token.position.line == closing_bracket_position.line and token.position.column == closing_bracket_position.column)) \
        | map(lambda token: token.value)

    return ' '.join(body_tokens)

In [31]:
tokens = list(javalang.tokenizer.tokenize(code_snippet))

get_method_body(tokens, line=53, column=16)

'{ if ( obj != null && obj instanceof CompositeCacheKey ) { CompositeCacheKey s = ( CompositeCacheKey ) obj ; return parserName . equals ( s . parserName ) && userAgent . equals ( s . userAgent ) ; } return false ; }'

In [32]:
def get_attributes(tree_node_tuple):
    tree, node = tree_node_tuple
    body = get_method_body(tokens, line=node.position.line, column=node.position.column)
    return body

In [34]:
val = tree.filter(javalang.tree.MethodDeclaration) \
    | map(get_attributes)

list(val)

['{ return cache . get ( new CompositeCacheKey ( parserName , userAgent ) ) ; }',
 '{ cache . put ( new CompositeCacheKey ( parserName , userAgent ) , details ) ; }',
 '{ if ( obj != null && obj instanceof CompositeCacheKey ) { CompositeCacheKey s = ( CompositeCacheKey ) obj ; return parserName . equals ( s . parserName ) && userAgent . equals ( s . userAgent ) ; } return false ; }',
 '{ return Objects . hash ( parserName , userAgent ) ; }']

In [9]:
tokens = javalang.tokenizer.tokenize('System.out.println("Hello " + "world");')
list(tokens)

[Identifier "System" line 1, position 1,
 Separator "." line 1, position 7,
 Identifier "out" line 1, position 8,
 Separator "." line 1, position 11,
 Identifier "println" line 1, position 12,
 Separator "(" line 1, position 19,
 String ""Hello "" line 1, position 20,
 Operator "+" line 1, position 29,
 String ""world"" line 1, position 31,
 Separator ")" line 1, position 38,
 Separator ";" line 1, position 39]